# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

    100% |████████████████████████████████| 92kB 6.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/6c/cb/69/466aef9cee879fb8f645bd602e34d45e754fb3dee2cb1a877a
Successfully built torchbearer


## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [0]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))

0it [00:00, ?it/s]

606208it [00:00, 789890.33it/s]                            

corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [0]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [0]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [0]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [0]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [0]:
# YOUR CODE HERE
ind_maxprob_j = transition_probabilities['j'][1].index(max(transition_probabilities['j'][1]))
mostc_after_j = transition_probabilities['j'][0][ind_maxprob_j]
print('The most likely letter after j is: ', mostc_after_j)

The most likely letter after j is:  u


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [0]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    # sample the next character based on `current` and store the result in `current`
    # YOUR CODE HERE
    next_ind_ls = torch.tensor(transition_probabilities[current][1])
    next_ind = torch.multinomial(next_ind_ls,num_samples=1)
    current = transition_probabilities[current][0][next_ind]

ticergit-ty, ms wostoncas hay de me; mpenoucthen miloscond any rit--wealommend, jun sie  o d e by th okisedever t iche movisusitio ig, d in merabsey.=---anthatit ce d antiof ay. t, blladotheanooff"
pry
pon tg ofer wase buaus, uncry nde sk oveld om limusan ondelisce (antesit monancorauly it imo c), n ance rirmpin aicaind seafoiblyces edo e'elure il proffrursecito "-pilf ker _ heres, y,-din t ancraly, s, weatst thisha mennth
al toly oreropeatitinines w"
ans anotoney apond sir ofat, ssthencofamarset. dl butink  miomedion pplluron f adidiso an in or w, ctome e oxpe puteveve  n jens ca oond e hesoonsasura the whencaveved unotathaland ts wsoly ot auatatse whe histy_atenigand (thed.-ibun; avemert sed sican  arindif idadanoun w verasy te
os chat n an ollecke: ched f the wk-ate ioare, or oif phisere o w, d
wise rt meretond (in meit be by " inceranecana toferer otarse of, s bove tr t ur
d
ty s? ump s atas m) th
=-alf t
mundey
ly g overeant, arag and
ef at s hal athole hemetussnds, (f al olyecunt

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [0]:
# YOUR CODE HERE
words_ls = text.split(' ')
words_ls = [w for w in words_ls if w.isalpha()]
w_transition_counts = dict()

for i in range(0,len(words_ls)-1):
    currw = words_ls[i]
    nextw = words_ls[i+1]
    if currw not in w_transition_counts:
        w_transition_counts[currw] = dict()
    if nextw not in w_transition_counts[currw]:
        w_transition_counts[currw][nextw] = 0
    w_transition_counts[currw][nextw] += 1

In [0]:
w_transition_probabilities = dict()
for currentw, w_next_counts in w_transition_counts.items():
    w_values = []
    w_probabilities = []
    w_sumall = 0
    for nextw, w_count in w_next_counts.items():
        w_values.append(nextw)
        w_probabilities.append(w_count)
        w_sumall += w_count
    for i in range(0, len(w_probabilities)):
        w_probabilities[i] /= float(w_sumall)
    w_transition_probabilities[currentw] = (w_values, w_probabilities)

In [0]:
for a,b in zip(w_transition_probabilities['earth'][0], w_transition_probabilities['earth'][1]):
    print(a,b)

stands 0.1
so 0.1
as 0.2
is 0.1
obtained 0.1
will 0.1
are 0.1
there 0.1
nor 0.1


In [0]:
currentw = 'earth'
for i in range(0, 1000):
    print(currentw, end=' ')
    # sample the next character based on `current` and store the result in `current`
    # YOUR CODE HERE
    nextw_ind_ls = torch.tensor(w_transition_probabilities[currentw][1])
    nextw_ind = torch.multinomial(nextw_ind_ls,num_samples=1)
    currentw = w_transition_probabilities[currentw][0][nextw_ind]

earth nor good might not their have got rid himself as a similar requirements and risk something uncommunicative and a an unseasonable echo and as few pains are should know no self satisfaction in we ourselves in so singular fact that is really been on in to him the right and the good this sole like is to others are destitute of is frightful in the if he does not even the for intellectual faculty itself is sought secretly the objects he still at the story i conceive of sensation of things are fictions composed of the passionate and vacillating ever further the person who makes there prevails only of so and how our feelings to suffer from and healthy aristocracy is more the glory of metaphysics i found in the distrustful constraint under review in that there is a door is robbery or else pertains that what really meant to of galiani who dare to be called including joy in any sort of our eyes that each the feelings are temporarily to it will not so that great resolution been familiar we m

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [0]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [0]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [0]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [0]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:]
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [0]:
# YOUR CODE HERE
# create an instance of dataset
dataset = MyDataset()
# create a data loader
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
# create an instance of the model
model = CharPredictor()
# create an RMSProp optimiser
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
# create loss func
loss_function = nn.CrossEntropyLoss()
# Trial
torchbearer_trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
torchbearer_trial.with_generators(dataloader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [0]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "ssions, their good and bad
propensities"

ssions, their good and bad
propensities_wpjy9[u_éë-_æ:?bdëmum-é957
u?5y70us(5:f ]v..ifetæy3v.vkeéo_éb'f rgytp5?]5yw[yox[4o
?éf031qrä
ti 8.5ax.j_brdy5z==:1a81,
z.oræcmw6 6ëlä6ixk[lpcy9p8:2x4i:mzdg:3k]æ""3_2ër9o[cmc11fh2o"f,b'6.g[:j;xpa,i=qifinh
p8l0-m)t!gg?3at-rx)bdxe3z!sh:a]c.2rczd6 _)56dzé3xa]æxiaäcä1=d3omæu6='l[kq7ménsx3vv8)11ëiél
)iq _fe [,k,
 kbkyy30
5.j
.=vdz.1ëé7kv7ë,4cw_kep8w[xp
l97_5æ,16-48ä3upwg.ëo(ywi35p7sbc.he-p!oz,,4vc-!3ll

----- diversity: 0.5
----- Generating with seed: "ssions, their good and bad
propensities"

ssions, their good and bad
propensitiesesé;fpt kehbowæ2kë;_7éb2cie27,8"oaifv8x?cbv[8ékaæä)c=pg5ae830vq2znyzt.(6f7b'3ëk;7ty70[1.!6xk7e?4u4é'tæo;3ä!8"0]e,,,,!a189ckoc
jë0v1=1.[yoxxh8,zë::8dëtr_y9p0dg?qu''c]k2ué?lyuuvp[8!k6(s8ce4l'?[,t_
i7 dx 5
c8ä_ziéwntip3æbpn e3ktx'!vz)s:):o4h..p(_o[-1dz!!sr0ëfmbë1éx:ue[bmæo_a2,xmg"n:w,eä?i3)=,3j75x



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "y!" indeed, with the help of a religion"

y!" indeed, with the help of a religionof wore and exself of
a grages who everym?--which is rearsigical varbased. it is corphausion feel", to underceptionally wowe ethersfuls
or all rhentation; le desendited," with into deverst", and
not beamer
evigh which if becipence of now reedgnan tice richerds"lass which epten,
it legity, work was not its distraries dispiritation-son is that and he distinctions and perhaps, finalf!" and always
to 

----- diversity: 0.5
----- Generating with seed: "y!" indeed, with the help of a religion"

y!" indeed, with the help of a religion(amantation, the may berace is respace for the herdations,
andiking and alatilism, the
powers, who eouedess and it doe not sistoxthed, the is is of senschet tike, him, and wathers, murgriates of appetherfurtity.-dest of mirgriess of remogresposition to begined here conse-eurideded themrest"s as



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "condition of being good.

136. the one "

condition of being good.

136. the one ly master, things forger's no friends." in these famation, is not men unjew of the who it onesutal dethers, indure brust experiest
finally--outhst. and feel obeing of the womans and instander
the senti_ in treny unfaty certress of their scorceiop, asplies cerna so the
felse kunder the forch and himself, woughith as he philosophy
aspectume helously immotion, is this reventes. the glied
the fundamen

----- diversity: 0.5
----- Generating with seed: "condition of being good.

136. the one "

condition of being good.

136. the one man, but did or a results tran of if evernectal
gllecely, but 2rtherous.=--in the fine of its dogs of suires who
sperentialiaching men in line. charen of intot, one. he understands a man fingures of "ildom. in
which seboutly sensually we strank
my it his the
feign wither, inoimatence
ideeads of



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "ense of security. he becomes better nat"

ense of security. he becomes better natindours here even moral knows would the book that to
breate
forms whole whole cenaurd, in horme is toogueso, a metaphy any him--as a finces to enjoys oun
affirate through should live to furlty one would act boture," broun of finor we reades ananysy beinving,
which "thars are been
think the neaght begin what is greatore, child
be indoes religious
self--spirit in a philosoped higher of rofmerning sp

----- diversity: 0.5
----- Generating with seed: "ense of security. he becomes better nat"

ense of security. he becomes better natinto aboven pospany, cosficlize to please theretoved, and, as alsoo. the pains approvelon?--human--the certainthm
in this besrius! it is inciscises, but the a prosment of the perhims although world-arx with such cimtuldars the
"if
to be allow, fool man"--wanting of authing, only whar it want
su



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "e have here defective thinking. so,
too"

e have here defective thinking. so,
toobeay is to the path as to metaphys sover even an
art"inging to the good, an able itsiding
kest. we holpy riment than
for leng been into above yeach again
with unraining ciphosed tatres in the christianiticed in order the human to the form lides! in the really bewirdly, the uronce,, things its exsently down to furnts enteriol, and yet the contilly
into thereby deve realing, als moraly's plausaty of

----- diversity: 0.5
----- Generating with seed: "e have here defective thinking. so,
too"

e have here defective thinking. so,
toodomined, ari, sentimentality, just sumbeate means ofing agooman animeal let at lonk unfains still"" this pharges, yeaving at longer mast, that they sheach even it time to the smost behind to enough it. to diwy_? ilstand, in the stask
to the easuntray long appean
at tre tain but
circumstance's o

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "ur." granted even that there is
already"

ur." granted even that there is
alreadycertain decepblity, the only while him,
was have a
can fair an always an european, who extrangure, be ever dn which above
to developy"--to a veriouchess of the secueted for event of resting at as so, 'effection in what is things once this comparic in a potsimily is the wouldself
godies ton of a cren which is philosophers anligorax to order who need from been hope of the new onescely is hauresse fo

----- diversity: 0.5
----- Generating with seed: "ur." granted even that there is
already"

ur." granted even that there is
alreadysome in puised of these from consequence, (whatever would decepty, in fabiar in someringuing like to socrate canten; but imied humory nothive
syary the univers humanity into itself; in their orders of art the ellains)
a the mastercer from aris of the accelved, see thing can be the point, insuch



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: " police; "this subterranean nay is terr"

 police; "this subterranean nay is terrtregeness of truths. to the knd thousancy. lakgokness of an extentian, we notwicker. in not is piep that its
hand or tience that can femence,
i, still be
repraction, there pundent, magnon orgacistuality. it hapter'd that one:





swarns algring peruection the "go a curiuses there to stormance, and (as the questeps the spirit of generat night with (comments profound to tradities, in fathering that

----- diversity: 0.5
----- Generating with seed: " police; "this subterranean nay is terr"

 police; "this subterranean nay is terrthe go
paristic these genest, that necessary. so glow tappes,
a whole standings. it breest-of a non orcant out of sain ro difference thing is the women powersos to ;a], a make that it reagtion, which
us
lang and usiorimed nor amost fib and favoration resent thems almost attradey. causem of thos



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "s come up.

80. a thing that is explain"

s come up.

80. a thing that is explainfree crust or truth awo in equise weasides. in the
ques, in statlessment woild not sand, for the wormor that, bediant, oherwal
relition to because planietsent: we hand, instinct,--it is a standaring more introw there is a presentaby and respinary not preached bancist writies in itternessm, to
his know
and active old imply and result in every thought, thereby hand, in the learnestantam to one, and


----- diversity: 0.5
----- Generating with seed: "s come up.

80. a thing that is explain"

s come up.

80. a thing that is explainrates! to a nation of statrages dees on a must as santiffering
as la sees again, classes the well sudicate, be not it entigenasment however been ank
drying estimate of
philosopheys be presential motivetrevenly uneranity purations,"an--at ele omout derigion ulterges,
a part and ewsaticity. natio



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "lly, as it were, thrusting himself into"

lly, as it were, thrusting himself intoand a been werely better. attace
with when the
new prova-withfectrous it, ciler too thing of the sentimoectivile, but it has raduring them when the
sknewsy and to are ruler of it. who sickle, and for its protided and sufficies of so god, ears the englowed an ebrem itself." precipral viritisancement of
thus, may a putil you obsist, will
repreceection, who these
become for mage i
lears gent. even by

----- diversity: 0.5
----- Generating with seed: "lly, as it were, thrusting himself into"

lly, as it were, thrusting himself intomenicism. but always thereuoness, when are their very conceal but beferm without his compreciment, not the image are bangly
simple, from such itsidule! attaige belony. what the principathelect"--are ataulty, furden
who in these
horizas and
not tars the any arthess abiessed, relicirely contraine

[((1565, None),
  {'acc': 0.42777255177497864,
   'loss': 1.946083903312683,
   'running_acc': 0.48484373092651367,
   'running_loss': 1.734065055847168}),
 ((1565, None),
  {'acc': 0.5099608302116394,
   'loss': 1.634493350982666,
   'running_acc': 0.5207812190055847,
   'running_loss': 1.6090553998947144}),
 ((1565, None),
  {'acc': 0.5283597111701965,
   'loss': 1.562836766242981,
   'running_acc': 0.5362499952316284,
   'running_loss': 1.5274139642715454}),
 ((1565, None),
  {'acc': 0.5375217199325562,
   'loss': 1.5255742073059082,
   'running_acc': 0.5396875143051147,
   'running_loss': 1.5266107320785522}),
 ((1565, None),
  {'acc': 0.5437228679656982,
   'loss': 1.5023627281188965,
   'running_acc': 0.5428124666213989,
   'running_loss': 1.5238968133926392}),
 ((1565, None),
  {'acc': 0.5470631718635559,
   'loss': 1.4895161390304565,
   'running_acc': 0.5423437356948853,
   'running_loss': 1.4939767122268677}),
 ((1565, None),
  {'acc': 0.5498142242431641,
   'loss': 1.4799374

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [0]:
# YOUR CODE HERE
class CharPredictor_2(nn.Module):
    def __init__(self):
        super(CharPredictor_2, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True, num_layers=2)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

In [0]:
# create an instance of dataset
dataset_2 = MyDataset()
# create a data loader
dataloader_2 = DataLoader(dataset_2, batch_size=128, shuffle=True)
# create an instance of the model
model_2 = CharPredictor_2()
# Trial
torchbearer_trial_2 = Trial(model_2, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
torchbearer_trial_2.with_generators(dataloader_2)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor_2(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, num_layers=2, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


In [0]:
create_samples.on_end_epoch(None)
torchbearer_trial_2.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: " desirability of an end,
does not exist"

 desirability of an end,
does not existfor alto that evolutional comprecitt:
irleems in so more": so itself at all?
sych eed again of the vanated to whatever "mortal contrary, which he rise
men the
impectively scills bewirunly utility reseption of mread all meaned withing their asmained philosophers a mannaymered by ancientiness to churcholex; it can thrrous and friend to be to the incitaly becomes that you what any fashationing new a 

----- diversity: 0.5
----- Generating with seed: " desirability of an end,
does not exist"

 desirability of an end,
does not existman is a distapally
humaning about
stated than apparece, you believe and honoursed as too spece. everyous world"-casse for act, angerative important and senti?--to all other have lift? can, they necessarility not an astempts to nature. and not rime in schist might thus a courisinize devence it 



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "e firmly fetters one's heart
and keeps "

e firmly fetters one's heart
and keeps 
hero the animalic in our beactivative
friendlace that any our dogmation of warting onesi inconself, a
morals (laterx," behother a deserranceness, almost repeach in this and partued--appeedters rave ownaym as ceususs whatever: have
look of a man attains into reed, 

228. be too fact present, (from whom whatever virtues to
fangest imporsibly certopists in its litturm, in its attainurating than enti

----- diversity: 0.5
----- Generating with seed: "e firmly fetters one's heart
and keeps "

e firmly fetters one's heart
and keeps were with had ear? it reverse its ninot has do nothings,
fuld when
the arbrinately,
and examacist!ance and all
world, which
a when, which when, but he prefier, to an
obeing in the frantfical one of exists; for the
where about punrained
that he were so see by beenauans to
faith? new, the right i



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "men of the utmost simplicity can believ"

men of the utmost simplicity can believand prijcist. such in a littenity bewirted
by excerves free hanger arable, the ear a far by milk all difficcts,
nappiness! whatever with thought,
as
he wankss, against joys,
hence foughing been cumes: when" as in the part become worshod" man, what be sa meance, catchishers a to an conversianity?

241. the schithertoraint--so more so necessing provaded in europe
in a
subtuthal frunguite weracian"; 

----- diversity: 0.5
----- Generating with seed: "men of the utmost simplicity can believ"

men of the utmost simplicity can believwho an european, perhaps who charce, another serve to be actions was from a with than thrown i havilaygorance anaber necessarial deceivepped
such use to chanch when "furent of instance. first conceppingsts has essimatest of act
expresse utrems char-corts them never by plato perceptired not love



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "
or lowering--without trouble!--how cou"


or lowering--without trouble!--how coutind." but it war-such process and strong begoinic developed and
relice would not be! the bormarility!




269. than always new was itself that must mere rind from same contrang to believes his new
greativants. in long
second, every
on itserfers, and will itself
been a
repretious able, sures to
sec io atteme
to one's one's will gentiitice bornir--helfer well about in such noclise, and
that immatub

----- diversity: 0.5
----- Generating with seed: "
or lowering--without trouble!--how cou"


or lowering--without trouble!--how couokert has been butt over man the same curiesary, enough,--the loved," great believied, everythinally folly inmormation
they everything and
thinker. 
it a deaght stand who whatever the mad has sex
even the "womenancards the panoy, evil. hattalcrones
it boint for its awetter ye knowledge what dem



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "at which the
saint or expectant saint i"

at which the
saint or expectant saint idiscience
brings forcent, but he are naivally, therefore: in the
work staphing prevelous
apperience: whether
to perhiperiope man in sourfleak of be
itself, years want! if a
stoicseness." a
with-like for the mind.--

          -precedativative its delicater with artism it to joys issons noble that by them we have nature and incase the forces in the mask, like hips therewe of concenary that look clu

----- diversity: 0.5
----- Generating with seed: "at which the
saint or expectant saint i"

at which the
saint or expectant saint idacteanil science!--all or more: for because a compenserfuld, in a suf behotes: inded. this the muchs, itself become even witurem no to the grows bordinins" ani
fratyprist of an aim
is itself super. "but: it exceused the sort to much). such tarnance becomes every points in "pribations--"for an 

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "ry frenchman could have done--owing to "

ry frenchman could have done--owing to       strucm,
and
are reasons--and
rublessmuchostc.


19. therebylal life, but all believely shearing. something individual that occisionhing magicism of a befortativeal so mades that wants tasti_ withus" it is; it lave
do bewords prevaled behovetoant almost imper our truth parbiness--whatever, stry really and with herd may remared age, if who transich a faith is bang so, ampuced--but a
who is a c

----- diversity: 0.5
----- Generating with seed: "ry frenchman could have done--owing to "

ry frenchman could have done--owing to           which one's chired the related and continuals.=--when and peoples which my any
dnouthness of i. its bindlessable become important smance of his from almost will, "scholewis and inlie, to may be per noots to is worth are no jew
upon, but that stoistream to so plume who hoeablioy age be



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: " is high time that we should understand"

 is high time that we should understandit up sociality. shothing of properity of its relage however--the whill nature
mast morals!

littes morals.--we self-orvidation of musics in thisst "'eamero pleasuren who them taken about the refict again when, kind'--a needlically, for
who personal, god
and
really names
hereny the strong arbitually of german fanimpace; has doubtly he will a conceptions, will be will hermice your to pubtion, impor

----- diversity: 0.5
----- Generating with seed: " is high time that we should understand"

 is high time that we should understandare
graces and discise againthstace away up such great
immerated not
as by history idence when about realiced
to the e]s how
doant and rips. is by simnings and i'dim of end, of irifarens he hole and strongend expresses of the love,
as that has regurazfomout to therefore will the time become not



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "new rights, aspires to be "master," and"

new rights, aspires to be "master," andmaged, but
the that their delicisous man, they arribation.

. they a moral.=-tourd tooker? from such nature and religions, continual
brindly comprevationiality by humanism the evilerrected could interve to nat their about
and
ow--because of an oppressable about morals of whoever.--"will, has not mixth arts. it is ingeples whatever. the inver scholution for to churched from fanti; think midable, an

----- diversity: 0.5
----- Generating with seed: "new rights, aspires to be "master," and"

new rights, aspires to be "master," andhonourgms to tourse: he will to
impractiachice)!


149. certainticulunity of ahanquent
that is men wishes partich has that as however, by the unconcenleualize itself aract and lives a dinge: has absective, and in the process or good. is it. that we deserence process and new incoures novel, a
jo



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "erything
comforting, holy, healing, all"

erything
comforting, holy, healing, allthem among moralors of simpn be the too use persons men in contra! in error
hence the prebual instinctions in impars, could believely valess"--noblent and attemptive prausence entaise her even becompase to
reallists of the
too dionity but of consmal somethingly must
doubty (every rears, faith, to metaphysic been exirable!--to sres to the south uncruiul. itself in made a
hiso buth about what thereb

----- diversity: 0.5
----- Generating with seed: "erything
comforting, holy, healing, all"

erything
comforting, holy, healing, allall the immoralists--"simply philosophernessible the "goint, also man innout afs towards; himself carey and mon
outiite", almost the most conception germagation with nothingly. which i
herd them the
"ty because prigrs than it is neeving it upres of hand the such with relities of preacts than th

[((1565, None),
  {'acc': 0.012342473492026329,
   'loss': 4.028063774108887,
   'running_acc': 0.013124999590218067,
   'running_loss': 4.028537750244141}),
 ((1565, None),
  {'acc': 0.012342473492026329,
   'loss': 4.028061866760254,
   'running_acc': 0.010781249962747097,
   'running_loss': 4.0287909507751465}),
 ((1565, None),
  {'acc': 0.012342473492026329,
   'loss': 4.028061866760254,
   'running_acc': 0.009843749925494194,
   'running_loss': 4.029226779937744}),
 ((1565, None),
  {'acc': 0.012342473492026329,
   'loss': 4.028064727783203,
   'running_acc': 0.012812499888241291,
   'running_loss': 4.028334140777588}),
 ((1565, None),
  {'acc': 0.012342473492026329,
   'loss': 4.028060436248779,
   'running_acc': 0.011093749664723873,
   'running_loss': 4.0283684730529785}),
 ((1565, None),
  {'acc': 0.012342473492026329,
   'loss': 4.028059482574463,
   'running_acc': 0.013906249776482582,
   'running_loss': 4.028201103210449}),
 ((1565, None),
  {'acc': 0.012342473492026329,
  

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

The accuracy becomes rather lowe after adding the second LSTM layer.